<a href="https://colab.research.google.com/github/Trickshotblaster/nn-practices/blob/main/TrigramLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [95]:
import random
import math

In [96]:
import requests

url = 'https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt'
url = 'https://raw.githubusercontent.com/karpathy/makemore/master/names.txt'
#url = 'https://raw.githubusercontent.com/Trickshotblaster/nn-practices/main/TrigramLM.ipynb'
response = requests.get(url)

with open('file.txt', 'wb') as f:
  f.write(response.content)

In [97]:
with open('file.txt', 'r') as data:
  dataset = data.readlines()

with open('file.txt', 'r') as data:
  raw_data = data.read()


In [98]:
dataset = raw_data.split("\n")

In [99]:
dataset[:10]

['emma',
 'olivia',
 'ava',
 'isabella',
 'sophia',
 'charlotte',
 'mia',
 'amelia',
 'harper',
 'evelyn']

In [100]:
for i, data in enumerate(dataset):
  dataset[i] = "." + data + "."

In [101]:
dataset[:10]

['.emma.',
 '.olivia.',
 '.ava.',
 '.isabella.',
 '.sophia.',
 '.charlotte.',
 '.mia.',
 '.amelia.',
 '.harper.',
 '.evelyn.']

In [102]:
raw_data = ''.join(dataset)

In [103]:
raw_data[:10]

'.emma..oli'

In [104]:
train_set = dataset[:math.floor(len(dataset) * 0.8)]
dev_set = dataset[math.floor(len(dataset) * 0.8):math.floor(len(dataset) * 0.9)]
test_set = dataset[math.floor(len(dataset) * 0.9):len(dataset) - 1]

In [105]:
train_set[:10]

['.emma.',
 '.olivia.',
 '.ava.',
 '.isabella.',
 '.sophia.',
 '.charlotte.',
 '.mia.',
 '.amelia.',
 '.harper.',
 '.evelyn.']

In [106]:
dev_set[:10]

['.amrom.',
 '.aniket.',
 '.ansen.',
 '.apolo.',
 '.aqib.',
 '.aquarius.',
 '.araf.',
 '.arafat.',
 '.areeb.',
 '.ariez.']

In [107]:
test_set[:10]

['.arib.',
 '.arinze.',
 '.aristeo.',
 '.arlind.',
 '.armahni.',
 '.arnez.',
 '.arnie.',
 '.arrie.',
 '.arris.',
 '.arrison.']

In [108]:
vocab = set(raw_data)

In [109]:
vocab

{'.',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z'}

In [110]:
vocab_len = len(vocab)

In [111]:
vocab_len

27

In [112]:
stoi = {char:i for i, char in enumerate(vocab)}

In [113]:
itos = {i:char for i, char in enumerate(vocab)}

In [114]:
import torch

In [115]:
import torch.nn.functional as functional

In [116]:
# make dataset
xs, ys = [], []
for data in train_set:
  for ch1, ch2, ch3 in zip(data, data[1:], data[2:]):
    xs.append([stoi[ch1], stoi[ch2]])
    ys.append(stoi[ch3])

xs = torch.tensor(xs)
ys = torch.tensor(ys)
num = xs.nelement()
num

314304

In [117]:
ws = torch.randn((vocab_len*2, vocab_len))
ws.requires_grad = True
ws.shape

torch.Size([54, 27])

In [118]:
xenc = functional.one_hot(xs[0], num_classes=vocab_len).float().flatten()
xenc @ ws

tensor([ 1.0992,  0.1850,  0.3321, -0.1491,  1.8630, -0.0954, -0.3334, -1.0885,
        -0.5622, -2.5288,  1.9680, -0.9247, -0.1170, -0.8643,  1.0266,  2.5070,
         0.6645,  1.2592,  1.9048, -2.3184, -3.8863,  0.9839,  0.6910, -1.3335,
         0.6830, -1.6274,  1.9658], grad_fn=<SqueezeBackward4>)

In [119]:
(xenc @ ws).shape

torch.Size([27])

In [120]:
num_epochs = 100
lr = 20

In [121]:
torch.randn((3, 1))

tensor([[ 0.9332],
        [ 0.1708],
        [-0.4885]])

In [122]:
for epoch in range(num_epochs):

    xenc = functional.one_hot(xs, num_classes=vocab_len).float().flatten(start_dim=1)
    logits = xenc @ ws
    counts = logits.exp()
    probs = counts / counts.sum(1, keepdim=True)
    loss = -probs[torch.arange(int(num / 2)), ys].log().mean() + (ws.mean() * 0.01)
    print(loss.item())

    ws.grad = None
    loss.backward()
    ws.data += ws.grad * -lr

4.146651744842529
3.7447128295898438
3.4575295448303223
3.24652361869812
3.0967602729797363
2.991867780685425
2.913304090499878
2.8499600887298584
2.796975612640381
2.751849889755249
2.712947130203247
2.6790781021118164
2.649341106414795
2.623037576675415
2.5996170043945312
2.5786404609680176
2.5597541332244873
2.5426671504974365
2.5271384716033936
2.512968063354492
2.499985456466675
2.488046407699585
2.4770278930664062
2.466822624206543
2.457341194152832
2.448505401611328
2.4402480125427246
2.4325110912323
2.425243616104126
2.418402910232544
2.4119508266448975
2.405853033065796
2.400080919265747
2.394609212875366
2.389413833618164
2.384474992752075
2.3797738552093506
2.3752939701080322
2.371020555496216
2.3669393062591553
2.3630383014678955
2.3593063354492188
2.3557324409484863
2.3523073196411133
2.3490219116210938
2.345867872238159
2.3428382873535156
2.3399252891540527
2.3371224403381348
2.3344240188598633
2.331824779510498
2.3293185234069824
2.3269007205963135
2.324566602706909
2.32

In [123]:
xs[:10]

tensor([[ 0, 22],
        [22,  2],
        [ 2,  2],
        [ 2, 20],
        [ 0, 17],
        [17, 10],
        [10, 21],
        [21, 24],
        [24, 21],
        [21, 20]])

In [124]:
def forward(xs):
  xenc = functional.one_hot(xs, num_classes=vocab_len).float().flatten(start_dim=1)
  logits = xenc @ ws
  counts = logits.exp()
  probs = counts / counts.sum(1, keepdims=True)
  outchar = itos[torch.multinomial(probs, num_samples=1).item()]
  return outchar

def prompt(ins):
  out = ins
  for char in range(500):
    last_two = ins[-2:]
    xs = torch.tensor([[stoi[last_two[0]], stoi[last_two[1]]]])
    outs = forward(xs)
    out += outs
  return out

prompt("Rom")

'Romaeailniaw.iaas.ahfaaaeae.eeaaiainb.ceaaaaeaa.aaa.aiiie.oa.eerrasejea.scefen.eeeeiziaiansiaiioaiieaeaeaiiatie.iiiaacaeoyakef.aaf.ageiaaieaooaarxeiaviiei.a.jiiaaexywich.e.iatauex.aak.aiaaea.eiaairiwaiiiza.eaariea.a.iiaaii..iayibuseiaeemi.tia.atyaaa.a.iia.aeaaaieamisi.aais..itabieai.eqyiiaamraaiaweivsf..aieaaieagaameey.yiniiiiiebjwbe.a..eaaa.aaiegee.efieyayagiieyi.weaaareeaia.aieeeeoaiahraraebaaaafaiekaeirmfortaaati.yaiiaaiaa.aezyaeaa.oia.aa..ayiiaaiii..aataaiaiioia..artaoiae.aaaeinieeaacoa.yiaai.i'

In [125]:
# make test set
txs, tys = [], []
for data in test_set:
  for ch1, ch2, ch3 in zip(data, data[1:], data[2:]):
    txs.append([stoi[ch1], stoi[ch2]])
    tys.append(stoi[ch3])

txs = torch.tensor(txs)
tys = torch.tensor(tys)
tnum = txs.nelement()
tnum

39052

In [126]:
xenc = functional.one_hot(txs, num_classes=vocab_len).float().flatten(start_dim=1)
logits = xenc @ ws
counts = logits.exp()
probs = counts / counts.sum(1, keepdim=True)
loss = -probs[torch.arange(int(tnum / 2)), tys].log().mean() + (ws.mean() * 0.01)
print(loss.item())

2.4852092266082764


In [127]:
# make dev set
dxs, dys = [], []
for data in dev_set:
  for ch1, ch2, ch3 in zip(data, data[1:], data[2:]):
    dxs.append([stoi[ch1], stoi[ch2]])
    dys.append(stoi[ch3])

dxs = torch.tensor(dxs)
dys = torch.tensor(dys)
dnum = dxs.nelement()
dnum

38860

In [128]:
dxs[:10]

tensor([[ 0, 20],
        [20,  2],
        [ 2, 11],
        [11, 17],
        [17,  2],
        [ 0, 20],
        [20, 18],
        [18, 21],
        [21, 23],
        [23, 22]])

In [129]:
xenc = functional.one_hot(dxs, num_classes=vocab_len).float().flatten(start_dim=1)
logits = xenc @ ws
counts = logits.exp()
probs = counts / counts.sum(1, keepdim=True)
loss = -probs[torch.arange(int(dnum / 2)), dys].log().mean() + (ws.mean() * 0.01)
print(loss.item())

2.4921157360076904


Looks like it might be overfitting? The dev set results seem fine, but the test set loss it much higher than it should be.

In [130]:
ws = torch.randn((vocab_len*2, vocab_len))
ws.requires_grad = True
ws.shape

torch.Size([54, 27])

In [131]:
num_epochs = 100
lr = 20
regularization = 0.01

In [132]:
last_loss = 2.1

for epoch in range(num_epochs):
    dxenc = functional.one_hot(dxs, num_classes=vocab_len).float().flatten(start_dim=1)
    dlogits = dxenc @ ws
    dcounts = dlogits.exp()
    dprobs = dcounts / dcounts.sum(1, keepdim=True)
    dloss = -dprobs[torch.arange(int(dnum / 2)), dys].log().mean() + (ws.mean() * regularization)
    dloss.detach()

    xenc = functional.one_hot(xs, num_classes=vocab_len).float().flatten(start_dim=1)
    logits = xenc @ ws
    counts = logits.exp()
    probs = counts / counts.sum(1, keepdim=True)
    loss = -probs[torch.arange(int(num / 2)), ys].log().mean() + (ws.mean() * regularization * (abs(last_loss - dloss)))

    print(loss.item())
    ws.grad = None
    loss.backward()
    ws.data += ws.grad * -lr

    last_loss = loss.item()

4.039921283721924
3.69507098197937
3.445436954498291
3.2593672275543213
3.1189703941345215
3.011286497116089
2.927229404449463
2.860156297683716
2.805246591567993
2.7592620849609375
2.7200984954833984
2.6863322257995605
2.656931161880493
2.6311113834381104
2.608255386352539
2.5878732204437256
2.5695691108703613
2.5530240535736084
2.537978172302246
2.5242178440093994
2.511569023132324
2.4998884201049805
2.489056348800659
2.478975296020508
2.4695611000061035
2.4607443809509277
2.4524669647216797
2.444676637649536
2.437330484390259
2.430389881134033
2.4238216876983643
2.417595863342285
2.411686658859253
2.4060702323913574
2.4007256031036377
2.3956332206726074
2.3907766342163086
2.3861398696899414
2.3817079067230225
2.3774683475494385
2.3734092712402344
2.369518995285034
2.3657877445220947
2.362206220626831
2.358765125274658
2.3554577827453613
2.3522753715515137
2.3492119312286377
2.3462612628936768
2.343416690826416
2.340672731399536
2.3380253314971924
2.3354690074920654
2.332998991012573

In [133]:
prompt("bro")

'bronynsb.yndn.vnnhs.ghtanlnennnynhndwktns.daekexndkennaa.sslgslnec.e.myrmsyaq..nnus.o.nna..sdsnnikuhsxauewniuhexcanwhnnnxxnnbtn.ngt..tislniten.nax.dcnlcndrnnxnnh.tn.e.hzstvsn.m..smyssb.e.l.lvns.anbnnnnthnnenuen..nqsklssfntrnomn.nhs..enn.frk..nl.nspovdnnscmnno.vnsngy.ydvek.ktneisuense.amnntnlspud.olsranunnzlc.nnrcesdn..mnihvn.nrinasbunxnwh.nnk.qao.enallsc..heon.rsybinrdemndnpn..nhcbi.hsd.zcnsrg.nv.eeenxzde.qd.l.hhratnd.u.p.ennnn.cm.rcnetinhbrnnxs.eynn.nmsnban.nnn.aszlbupmtvalchmrnhvsfnmpewunr.nynepn'

Idk, maybe? I've definitely screwed something up here but I have no idea what

In [134]:
num_epochs = 100
lr = 20
regularization = 0.1

In [135]:
ws = torch.randn((vocab_len*2, vocab_len))
ws.requires_grad = True
ws.shape

torch.Size([54, 27])

In [136]:
for epoch in range(num_epochs):

    xenc = functional.one_hot(xs, num_classes=vocab_len).float().flatten(start_dim=1)
    logits = xenc @ ws
    counts = logits.exp()
    probs = counts / counts.sum(1, keepdim=True)
    loss = -probs[torch.arange(int(num / 2)), ys].log().mean() + (ws.mean() * regularization)

    print(loss.item())
    ws.grad = None
    loss.backward()
    ws.data += ws.grad * -lr



4.264211654663086
3.8671412467956543
3.5927422046661377
3.397332191467285
3.2511322498321533
3.133908748626709
3.0365138053894043
2.9548227787017822
2.8860952854156494
2.828001022338867
2.7785120010375977
2.7359535694122314
2.699002742767334
2.6666228771209717
2.6380109786987305
2.6125473976135254
2.589751958847046
2.5692529678344727
2.5507547855377197
2.534013032913208
2.5188217163085938
2.504999876022339
2.4923877716064453
2.480842113494873
2.470235824584961
2.460456609725952
2.451406717300415
2.4429991245269775
2.4351601600646973
2.427825927734375
2.4209413528442383
2.414458990097046
2.4083385467529297
2.402545213699341
2.397048234939575
2.3918216228485107
2.386842727661133
2.3820910453796387
2.377549171447754
2.373201608657837
2.3690342903137207
2.3650355339050293
2.3611934185028076
2.3574981689453125
2.3539414405822754
2.3505144119262695
2.347210645675659
2.344022274017334
2.3409435749053955
2.3379690647125244
2.3350934982299805
2.332310914993286
2.329618453979492
2.32701110839843

In [137]:
prompt('em')

'emmy.nnh.a.aa.ayeiq.iaa.iiak.a.eyaa.yipaiisiyyg.yooaaaiq.a.ayaai.aaaa.aia.gmdieaamxe.ih..ghdm..egi.ialaajiy.a..e.zayzamirdiaam.a.eay.aiqaigoaiaaapaaadgaaa.taiimaam.ai..sisi.uiiqoiayaiiai.wia..ai..amiaaa.ermaaeaooo.aaaqaa..ay.i.e.a.i.iuaa.iiysyasmdeiaiyiaoiediaaaiaay.ayxa.a.eaiima.ie.r.a.ekiai.caiaao.i.adeiy.cca.i..impaarayeraim.maaamx.ismsn.waoge.eikgxaaegi.aao..d..ai.a.airsaayoeaa.i.aoazig.oliim.eiiai.a.as.lhyiiayinaae.aaapa.eiipqikoi..giaiaaai.aegi.aiaimoaiann.ahmiemadkiaeiam.eaia.n.simamaiaylia'

Okay this seems promising...

aeoioa, vim, esiaota, aipe, seto, and ioa are all decent

In [138]:
prompt('em')

'emad.igtasa.yeeex.okosaa.eqaaaaaaeaa.i.r.imieiaia.cbaaeiiigl.as.iilaiakmd.ey.rii.eac..ipadiaamicamamiiea.poripceidiiaoaemasayimy.iiaaa.m.aaq.isiie.rmu.aa.ileekio.dmm.a..kcim..omaa.yakx.aaaigraaozsay.aaet..aeym.yapiisa.egaiz.e.yigaoitkieoipiski.axa.i.gikaiaa.aiee.m.m.y.mg.mieiam.g.wyaaia.mci.eapethmqatpdciyiaa.yiaoaaayiiomraqiyiaaad...a.sakyamdiiaiiediayiyu..oaiei.ke.r..a.aia.teairiire.ayiraya.ziiiiaioaegisime.amy.aoi.yeaiyiicii.y.lianieairaariaiidyyla.aaoihea.iaiiyeaeiaaikgeemma.jaeim.zpka.yoa.gle'

In [139]:
ws = torch.randn((vocab_len*2, vocab_len))
ws.requires_grad = True
ws.shape

torch.Size([54, 27])

In [140]:
prompt('em')

'emuill..w.fjau..xaxul.alxiaa.txli.xylqj..ip.ii..ilixa.uxxlaxa.xiray..i.jx.ufi.aixxailluuailyi..plx..xpa.xxuxlloxy.i.atilaat.r.xfi..slaauxvxili.li.l.ltaiauiza.ilyuufbu.iiaxxxigxiaxsxuiauxiiaa..xaxarxarxclkn..i.lnuiiaiaarabxa.ulxpau.xxxx.usulxiuiii.i.xiu.x.lxau.ukykuxal..b.lraxxaq..iuia.xpux.lxxyf.fx.xrupi.lpa..z.iiplxlfl.xi.pulxluqx.xxxxa.iiax.brlaluu.xxxaupi.fv.auitl..kxpugiaxxxlixllaxall..wa.a..bkal.luxwkux.u.kuy.vixlzaxlf..uf.fpax.lixixxia.izlixxaia.x.uvxaa.xsxxalva..ua.pur.f.eulxm.uialilx.iuxaa'

There's definitely some improvement over base randomness sometimes, but for others, it sucks

In [141]:
regularization = 0.05

In [142]:


for epoch in range(num_epochs):
    dxenc = functional.one_hot(dxs, num_classes=vocab_len).float().flatten(start_dim=1)
    dlogits = dxenc @ ws
    dcounts = dlogits.exp()
    dprobs = dcounts / dcounts.sum(1, keepdim=True)
    dloss = -dprobs[torch.arange(int(dnum / 2)), dys].log().mean() + (ws.mean() * regularization)
    dloss.detach()

    xenc = functional.one_hot(xs, num_classes=vocab_len).float().flatten(start_dim=1)
    logits = xenc @ ws
    counts = logits.exp()
    probs = counts / counts.sum(1, keepdim=True)
    loss = -probs[torch.arange(int(num / 2)), ys].log().mean() + (ws.mean() * regularization)

    print(loss.item(), dloss.item())
    ws.grad = None
    loss.backward()
    ws.data += ws.grad * -lr


4.240062713623047 4.196844100952148
3.829982280731201 3.853170871734619
3.539628744125366 3.6119472980499268
3.3373332023620605 3.4443278312683105
3.1902883052825928 3.3221781253814697
3.0764071941375732 3.227492094039917
2.9851863384246826 3.1519126892089844
2.910688877105713 3.090407609939575
2.8488216400146484 3.039299249649048
2.7967019081115723 2.9959874153137207
2.752281904220581 2.958686113357544
2.714054822921753 2.9261474609375
2.6808691024780273 2.897465229034424
2.6518216133117676 2.871958017349243
2.6261940002441406 2.8490965366363525
2.603412389755249 2.8284668922424316
2.5830161571502686 2.8097376823425293
2.564634323120117 2.79264235496521
2.5479681491851807 2.7769646644592285
2.5327744483947754 2.7625274658203125
2.5188560485839844 2.749183416366577
2.5060503482818604 2.736809492111206
2.494222402572632 2.72530198097229
2.483260154724121 2.7145729064941406
2.4730687141418457 2.704545259475708
2.463567018508911 2.6951537132263184
2.4546873569488525 2.686340570449829
2.44

In [143]:
dxs[:10]

tensor([[ 0, 20],
        [20,  2],
        [ 2, 11],
        [11, 17],
        [17,  2],
        [ 0, 20],
        [20, 18],
        [18, 21],
        [21, 23],
        [23, 22]])

In [144]:
xs[:10]

tensor([[ 0, 22],
        [22,  2],
        [ 2,  2],
        [ 2, 20],
        [ 0, 17],
        [17, 10],
        [10, 21],
        [21, 24],
        [24, 21],
        [21, 20]])

In [145]:
prompt("em")

'emadwmlesr.nie.sirbia.emaaexialio.yaa.eaufyeaieeawyomaios.yakga.a.iiaalyio..iaaaaaaje.ii.aqoniae.eiai..ximiaiaa..yoeaa.aeiiuaha.a.eoa.aea.yeaiaiv.lyexeiayaidoy.a..pi..vaigiiigi..aaiyapii.aie..giaeeba.aayamiamaiiigiiika.iiyza.i.a.iaaoiian.a.a.sieauzigi.iienlieyuie.ay.aiagaz..y.a.eaa.aiaaoiyaaiiisib.pm.uamia.oaeyb.ieainyiymaapa..fi.oieiiaaaace.aeaiaiif...aie.aiia.sia.aseaga.pmai.otiaegaaye.aiyaymie.amia.caa.s.lli.aemgaa.iai..yyaiiepaeimiimoaaa...io.ak.siomaaia.ai.iely.ssiiiia.i.ltgriii.h.iaadeeijain'

High regularization results in a fairly large disparity between normal and dev sets, but it seems to produce better results in odd cases

In [146]:
xenc = functional.one_hot(txs, num_classes=vocab_len).float().flatten(start_dim=1)
logits = xenc @ ws
counts = logits.exp()
probs = counts / counts.sum(1, keepdim=True)

loss = -probs[torch.arange(int(tnum/2)), tys].log().mean()
loss

tensor(2.4995, grad_fn=<NegBackward0>)

In [147]:
ws.shape

torch.Size([54, 27])

Wait

In [148]:
ws = torch.randn((vocab_len, vocab_len, vocab_len))
ws.requires_grad = True
ws.shape

torch.Size([27, 27, 27])

In [149]:
xenc = functional.one_hot(xs, num_classes=vocab_len).float()
xenc.shape

torch.Size([157152, 2, 27])

In [150]:
xenc[0].shape

torch.Size([2, 27])

In [151]:
(xenc[0] @ ws).shape

torch.Size([27, 2, 27])

Holup, gotta learn matrix multiplication again, it's hard being a 3 dimensional being trying to comprehend 700 dimensional math

In [152]:
2 + 9 + 25

36

In [153]:
a = torch.tensor([[2, 8, 3],
                 [5, 4, 1]])

b = torch.tensor([[4, 1],
                 [6, 3],
                 [2, 4]])
# each row/column is an n tuple
# dot product of (a, b), (c, d) is a*b + c*d

a @ b


tensor([[62, 38],
        [46, 21]])

In [154]:
inputs = torch.randn((2)) # inputs transposed?
print(inputs)


w1 = torch.randn((2, 1))  # (num in prev, num in target)
print(w1)
inputs @ w1

tensor([-0.0658,  0.4113])
tensor([[0.8495],
        [0.4958]])


tensor([0.1480])

In [155]:
inputs = torch.randn(10) # 10 inputs (same as randn(1, 10))

w1 = torch.randn(10, 3) # takes in 10, goes to 3 neurons
w2 = torch.randn(3, 8) # holy moly this is so easy bruh i was overcomplicating it
w3 = torch.randn(8, 1) # take in 8, to 1 neuron

wi1 = inputs @ w1
wi2 = wi1 @ w2
wi3 = wi2 @ w3
wi3

tensor([-15.0408])

In [156]:
xenc[0].shape

torch.Size([2, 27])

In [157]:
xenc[0] @ torch.randn(27, vocab_len)

tensor([[ 1.0453, -0.6560, -0.2914, -0.2650, -0.5257,  0.1919, -0.5012, -0.9533,
          2.4707, -0.4729, -1.8837, -2.3957, -0.9601, -0.0288, -0.2552, -0.4242,
         -0.7089,  0.8960,  2.2341,  0.2556,  0.4734, -0.8018, -0.7373,  0.0290,
          0.6066,  0.3971, -0.6067],
        [-0.7906, -1.4233,  0.9381, -0.2078, -0.6279,  0.6031, -0.1861,  0.7809,
          0.1411,  1.6500, -0.2014, -0.1855,  0.8052,  0.2001,  0.8525, -0.2062,
         -0.1582, -0.1472, -0.5068, -0.4175,  1.1444, -1.0875, -1.0663,  0.6578,
         -1.2933, -2.1694,  0.8118]])

ok maybe I wasn't as wrong as I thought. Other people might just be counting each individual trigram and doing a one_hot with num_classes as 27*27, while I just had a 'two-hot' tensor with the shape of (1, 54).

In [158]:
# make dataset
xs, ys = [], []
for data in train_set:
  for ch1, ch2, ch3 in zip(data, data[1:], data[2:]):
    xs.append([stoi[ch1], stoi[ch2]])
    ys.append(stoi[ch3])

xs = torch.tensor(xs)
ys = torch.tensor(ys)
num = xs.nelement()
num

314304

In [159]:
xs[:10]

tensor([[ 0, 22],
        [22,  2],
        [ 2,  2],
        [ 2, 20],
        [ 0, 17],
        [17, 10],
        [10, 21],
        [21, 24],
        [24, 21],
        [21, 20]])

In [160]:
xenc = functional.one_hot(xs, num_classes=27).float().flatten(start_dim=1)

In [161]:
lr = 10
num_epochs = 1000

In [162]:
ws = torch.randn(vocab_len*2, vocab_len)
ws.requires_grad = True

for epoch in range(num_epochs):
  logits = xenc @ ws
  counts = logits.exp()
  probs = counts / counts.sum(1, keepdim=True)

  loss = -probs[torch.arange(xs.nelement() // 2), ys].log().mean()

  ws.grad = None
  loss.backward()
  ws.data -= ws.grad * lr
  if epoch % 10 == 0:
    print(loss.item())

4.188051223754883
3.0435733795166016
2.741241455078125
2.598267078399658
2.515678882598877
2.4618256092071533
2.423388719558716
2.3943471908569336
2.371595621109009
2.353297233581543
2.338268995285034
2.325711250305176
2.315065860748291
2.3059334754943848
2.2980234622955322
2.2911183834075928
2.2850520610809326
2.2796928882598877
2.2749359607696533
2.2706947326660156
2.266896963119507
2.2634823322296143
2.26039981842041
2.25760555267334
2.2550625801086426
2.2527389526367188
2.250608205795288
2.2486462593078613
2.2468340396881104
2.2451536655426025
2.2435905933380127
2.2421317100524902
2.240767002105713
2.2394862174987793
2.23828125
2.237144708633423
2.236070394515991
2.235053062438965
2.2340877056121826
2.2331700325012207
2.2322959899902344
2.2314627170562744
2.2306666374206543
2.229905366897583
2.2291767597198486
2.228478193283081
2.2278079986572266
2.2271645069122314
2.226545572280884
2.225950241088867
2.225376844406128
2.2248241901397705
2.2242913246154785
2.2237770557403564
2.22327

Ok I guess I was doing it right the first time, but I'm glad I got the practice with matmul and neural nets :)

In [163]:
prompt("em")

'emip..a..aiyiymtamiaaoi.meiybb.ysma.ir..i.i..ii.aauaaiiemaamiypb..aoaribgabaaiobmiia.ie.aamm.y.agao..e.ic.ioy.kayii.gy.aoaieoeaa.eiva..a...a.i.iar.arer.ma.iaeaheaysf.eia..isa.avma.a.niyyqiaom.oeieaeabiecaa.k.goalia..aeia.a.eeibaaai.iya.amireta.aii.i.aaa.t.amaai.s.eiaisiieo..g.aoaea.i.aaaa..ai.ioaooaa..oa.aimioaieaaaaxiai.aiob.ieaey.aaoyiaaamis....iaimdyaa.ieyi.iaarb.ah.aaaiara.pa.emeaiaamepiybrai.eaaeaaa.aeawaaaagaaai.aimamaaagasimi.a.iyaaoare..iaaiaia.aieaadiia.a.eimipmaiaaaeyaaar..aicm.a.iyaoiyy'

In [164]:
ws = torch.randn(vocab_len*2, vocab_len)
ws.requires_grad = True

for epoch in range(num_epochs):
  xenc = xs
  logits = torch.stack([ws[x[0]] + ws[x[1]] for x in xenc])
  counts = logits.exp()
  probs = counts / counts.sum(1, keepdim=True)

  loss = -probs[torch.arange(xs.nelement() // 2), ys].log().mean()

  ws.grad = None
  loss.backward()
  ws.data -= ws.grad * lr
  if epoch % 10 == 0:
    print(loss.item())

4.42507791519165


KeyboardInterrupt: ignored

Yeah ig it works but it's unoptimized af right now... who cares, though?

In [185]:
# ok cross entropy time
for epoch in range(num_epochs):
  xenc = functional.one_hot(xs, num_classes=27).float().flatten(start_dim=1)
  loss = functional.cross_entropy(xenc @ ws, ys)
  ws.grad = None
  loss.backward()
  ws.data -= ws.grad * lr
  if epoch % 10 == 0:
    print(loss.item())

2.7844204902648926
2.7604000568389893
2.7383432388305664
2.7180495262145996
2.6993393898010254
2.682053565979004
2.666048049926758
2.651193380355835
2.6373751163482666
2.6124446392059326
2.6011581420898438
2.5905582904815674
2.580580234527588
2.571166753768921
2.562267780303955
2.553838014602661
2.5458383560180664
2.5382323265075684
2.5240793228149414
2.5174787044525146
2.5111634731292725
2.505112648010254
2.4993088245391846
2.493734121322632
2.4883739948272705
2.4832143783569336
2.4782419204711914
2.4688167572021484
2.464343786239624
2.4600183963775635
2.4558324813842773
2.4517791271209717
2.4478514194488525
2.444042921066284
2.4403483867645264
2.4367620944976807
2.42989444732666
2.4266045093536377
2.4234049320220947
2.4202919006347656
2.4172616004943848
2.414311647415161
2.411437749862671
2.408637285232544
2.4059078693389893
2.4006502628326416
2.3981173038482666
2.3956453800201416
2.393232822418213
2.3908755779266357
2.38857364654541
2.386324405670166
2.3841261863708496
2.38197684288

Bro ez ws

In [186]:
prompt("em")

'em.aiir.yi.e.aeimioropeemiinarri.im.dar.iioiibgoiaearaaba.la.eaii..aaaeikaacemiy.i.m.mobi.ibre.ain.a.ieiapi.ifia.l.a.ae.upa.ea.b.ibyabiaai.aooimmoarbammaniaai.ieo.eiaiymta.iiaiaaaeiimir..yae.eii.i.aaeasiirbaacioiaa..aime.s.ieieijaaoo.i.o.iiib..pybaami..myeaiaai..aii.c.aa.aaioiiea.e.daaj.meii..a.ae.aaaraia..aiemaamafaaaiy.akaaa.aaoo..aaiaoy..aa.epaai.ma..admmot.imiaros.yba.iaaa.i.aeai.iuiaaaec.i.ea..b.ia.a..i..iammyka..i.saaaa.iioaa.giaaaaai.aainiiaamt.ama.oiiaambiei.ia.mi.ai...aa.eagb..ai...rberaa'

ok now time for some multi layer stuff

In [199]:
w1 = torch.randn((vocab_len*2, vocab_len), requires_grad=True)
w2 = torch.randn((vocab_len, vocab_len), requires_grad=True)

In [203]:
lr = 0.05

In [206]:
for epoch in range(num_epochs):
  xenc = functional.one_hot(xs, num_classes=27).float().flatten(start_dim=1)
  wi1 = xenc @ w1
  wi2 = wi1 @ w2
  loss = functional.cross_entropy(wi2, ys)
  w1.grad = None
  w2.grad = None
  loss.backward()
  w1.data -= w1.grad * lr
  w2.data -= w2.grad * lr
  if epoch % 10 == 0:
    print(loss.item())

2.675579309463501
2.6710140705108643
2.6665289402008057
2.662121057510376
2.657789707183838
2.653531551361084
2.649346113204956
2.645230531692505
2.641184091567993
2.637204647064209
2.633291006088257
2.629441261291504
2.6256537437438965
2.6219279766082764
2.6182618141174316
2.614655017852783
2.611104726791382
2.6076104640960693
2.6041712760925293
2.6007862091064453
2.5974528789520264
2.5941715240478516
2.590940475463867
2.587759256362915
2.584625720977783
2.58154034614563
2.578500747680664
2.5755066871643066
2.5725574493408203
2.5696523189544678
2.566789388656616
2.5639688968658447
2.561189651489258
2.558450698852539
2.555751323699951
2.553091049194336
2.550468921661377
2.5478837490081787
2.545335531234741
2.5428237915039062
2.54034686088562
2.5379045009613037
2.535496234893799
2.533121109008789
2.5307796001434326
2.5284693241119385
2.526190996170044
2.5239431858062744
2.521726131439209
2.519538640975952
2.517380475997925
2.5152511596679688
2.5131497383117676
2.511075973510742
2.509029

In [210]:
prompt("br")

'brijeiiyaaiiiieiiiieaaeiiri.iiayyeiaaaeaiaaaaatiiiai.yiiiobaasliiloaaioiieiiaaadlaiiiiiaaiaaiioatuiaiatieiyceeiriyuioeykaeiaieieqeyaiaobieiaiyaououiaiiayeaeieooiaaetiiet.oaoillioieiiliaaailiyey.ileyaiteyauaherilyyaaeieoyeoraeiyeoaayaieyaaiiiaiyetibeiaeaioaieoeaeyeaieqoiyioe.eaaiaaearoieeaeeaiiaeeateiaaaoeioiuiooieriyiasylyiiiaiiyytisayeoaaieteleieaaiaiioiiaooaeaoeiliiuiiu.aieeyaiitiyaeiaaeiiybaaiaioaeyoaartauyiam.aeiyeay.eiiileaaraiioeeoiaiiioiiaaecoiuitiaeieiayeiiooieiiiiialaoiaiialheoiyiieaiiiuy'

In [212]:
def forward(xs):
  xenc = functional.one_hot(xs, num_classes=vocab_len).float().flatten(start_dim=1)
  logits1 = xenc @ w1
  logits = logits1 @ w2
  counts = logits.exp()
  probs = counts / counts.sum(1, keepdims=True)
  outchar = itos[torch.multinomial(probs, num_samples=1).item()]
  return outchar

def prompt(ins):
  out = ins
  for char in range(500):
    last_two = ins[-2:]
    xs = torch.tensor([[stoi[last_two[0]], stoi[last_two[1]]]])
    outs = forward(xs)
    out += outs
  return out

prompt("br")

'brhavarelpeveiieeieiekirrimoaamiefaiiauiaopea.ioeoeedezabaeu.oeipishwkelokkauxoie..xeikhreuellawiaoeuaradakpaaizeraoaaerenuiieaaseaeeuh.raaa.uaayoliaahoaaseehoinezoaoz.ay.eeiaaaarlsihieohioeeaeuoyiaaazzeiuiaioauehaofo.iie.een.raooaa.kjaoifeaorz.haaekaiiiihkk.headoiiaiiaeaoreeriieahoahoieatonkazroakakpoaeaalanaaiiaeau.earopoaroafi.ixoheuiianoeoaeiu.osaeueeakeaaaeieaoarnaeeahopodoiihelriaeedon..aeoezidoioknsadmeioheraa.rekeaioeaueaazeapezoaaeoe.saaieeaaolaokerraaoh.lauhooehntaoo.ozoeeoaealout.eiutip'

It ain't much but it's honest work